In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08981585502624512
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 35.0520704432684
200000 64.31104509354417
300000 60.99954581928679
400000 60.582676074606624
500000 60.08373962129645
600000 60.523130609659766
700000 71.23613699777289
800000 81.7417394958772
900000 82.57390391212085
1000000 85.87289763152789
1100000 95.01028404096179
1200000 114.62856325317387
1300000 32.28667356416298
1400000 105.77210341027475
1500000 125.49887000803821
1600000 116.04579079588389
1700000 107.3399688200861
1800000 111.63955501840393
1900000 109.43096270336235
2000000 107.75853968158452
2100000 103.58168208167088
2200000 105.61549667552187
2300000 107.2033744797149
2400000 105.57056227813588
2500000 94.99268785984445
2600000 31.676864257565615
2700000 106.94260615809047
2800000 105.9171471804559
2900000 100.7782753609371
3000000 106.08

29400000 106.89980344778195
29500000 104.05384180871833
29600000 106.23103891917361
29700000 106.99171831080902
29800000 105.6256404392553
29900000 104.48080072204891
30000000 65.24538712107469
30100000 87.78628940366335
30200000 109.60912368399545
30300000 115.31551510578198
30400000 91.01914744075947
30500000 67.81762816307398
30600000 118.14357944117445
30700000 113.03850339718487
30800000 116.39607456890764
30900000 113.3339772696958
31000000 111.70029792475489
31100000 112.11658096781942
31200000 108.1236216849388
31300000 107.38130283760054
31400000 107.20526595838015
31500000 20.342045921807465
31600000 111.42162356225928
31700000 105.26052012867866
31800000 107.03909520823022
31900000 109.58849685794367
32000000 105.04742864830023
32100000 109.43022060863777
32200000 113.5117625032754
32300000 114.19233462666817
32400000 113.30707865373003
32500000 64.52799898658866
32600000 92.66673531921653
32700000 116.43496756926294
32800000 114.59465678718135
32900000 115.13159593138624
33

59100000 75.41560231353152
59200000 45.98671046325604
59300000 89.41997253010979
59400000 90.60554349427737
59500000 96.04699081789148
59600000 98.26379948785059
59700000 99.35873871525608
59800000 98.64360715526398
59900000 99.03223906253899
60000000 37.00601256864492
60100000 114.48540148438443
60200000 107.47572799864324
60300000 103.67961667272571
60400000 106.9942560210358
60500000 110.90227705226926
60600000 104.2749062796282
60700000 103.04404211059857
60800000 91.56215335645905
60900000 51.029009234517275
61000000 104.5045657820421
61100000 102.34256752308777
61200000 103.0519344785156
61300000 102.8138930355415
61400000 96.15515874272482
61500000 96.5689845946792
61600000 94.49665026424486
61700000 29.323526036895913
61800000 98.50801379679842
61900000 96.2746059053156
62000000 96.53309320615763
62100000 102.63082337470746
62200000 99.03066842694275
62300000 104.73738547041341
62400000 105.61332182749787
62500000 35.35867307298108
62600000 112.61382413427131
62700000 111.32470

89200000 91.08107105962759
89300000 62.275196350827954
89400000 51.0691592593376
89500000 87.27410117099846
89600000 90.08526219355932
89700000 85.22224426794372
89800000 90.52261182750667
89900000 74.42968979442296
90000000 59.10747948470904
90100000 92.52835986511533
90200000 96.10503669332739
90300000 92.91365266341303
90400000 101.06558565456183
90500000 104.37861118063117
90600000 97.30623708923098
90700000 99.49708574017005
90800000 106.58389012290843
90900000 100.18477177018663
91000000 103.60492530784772
91100000 99.75517558709451
91200000 89.74032850817913
91300000 40.54737171350209
91400000 92.99516271205705
91500000 93.87887182900589
91600000 98.33983964762488
91700000 89.22253836591335
91800000 76.52331823207743
91900000 39.062968318782026
92000000 94.17304717924725
92100000 93.08997426577145
92200000 92.83213710258985
92300000 97.27537546247086
92400000 71.4993788856407
92500000 64.24576486908235
92600000 98.38263802082425
92700000 96.71465480515508
92800000 104.1516605046

118700000 102.11489361249659
118800000 92.65163819387773
118900000 94.72008791161758
119000000 91.84490694765378
119100000 65.49853279441629
119200000 60.717954931654596
119300000 87.61165855248098
119400000 90.54652794749784
119500000 50.14843257992687
119600000 67.41058697193384
119700000 84.18596736271034
119800000 93.45116158966218
119900000 51.97648214075594
120000000 80.3147543654562
120100000 90.5263138055134
120200000 88.41522253872877
120300000 89.44203375253917
120400000 94.73300822603241
120500000 102.60941728388697
120600000 95.73004933834787
120700000 103.15234764240367
120800000 102.67462141633125
120900000 100.47193008448126
121000000 103.58458491350265
121100000 103.77778702362654
121200000 96.88362388493513
121300000 97.52738115595663
121400000 97.16342438047512
121500000 93.22435994119432
121600000 93.41569891194632
121700000 33.08312797230611
121800000 94.13926505505923
121900000 91.70798168906913
122000000 65.29218856049987
122100000 53.497121220287376
122200000 89.

...

952800000 0.6208355253976683
952900000 1.116342231633323
953000000 1.5875848561410626
953100000 2.4964718442333758
953200000 1.9690278127963836
953300000 1.4049116174609488
953400000 0.7273710802905202
953500000 0.8226842927712752
953600000 1.5205907373196073
953700000 1.5648008328949454
953800000 1.5062704057391052
953900000 1.3764320973936823
954000000 1.5804335441483797
954100000 0.6373387536580603
954200000 1.4875954802524363
954300000 1.7621627688921868
954400000 1.8507744209104553
954500000 1.4220077554730595
954600000 1.2764699202984968
954700000 0.5611499319048279
954800000 1.27971233007414
954900000 1.4852934557205757
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.6264329458170632
955100000 1.3943486062310206
955200000 1.535552067436427
955300000 1.1848821684790862
955400000 1.3963502074862115
955500000 1.4133817738621435
955600000 1.564827560649275
955700000 2.558037621605896
955800000 1.6701298269140046
955900000 1.1156269270308143
95600000

979100000 0.4923344699877839
best so far: 0
type: [1, 2, 17, 1, 4] 136
cases of this type: 147968
979200000 0.3281688619596541
979300000 0.5071860669918888
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.3929706309815903
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.3215158218772232
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.8247476208515003
979700000 1.8663617876011818
979800000 1.7740452044185915
979900000 1.838672057217812
980000000 1.5996205360324847
980100000 1.5797746354656261
980200000 1.4551721415726888
980300000 1.7412673662908422
980400000 1.199781714159179
980500000 1.3554081279682333
980600000 2.167513396942468
980700000 1.5806332187869467
980800000 2.10683084489389
98090

998400000 0.8847714579004049
998500000 1.3913882728834928
998600000 1.5295895855211528
998700000 0.9889800740742961
998800000 1.4845494977812692
998900000 1.4470071500381767
999000000 0.9539050852251608
999100000 1.2845504124715852
999200000 1.4891205236805976
999300000 1.0719681211397647
999400000 1.0459978762185977
999500000 1.4212819124599112
999600000 1.5528163847181
999700000 1.1429615203482733
999800000 1.4094374115924297
999900000 1.3034199451367519
1000000000 0.84273596604612
1000100000 1.4270854159692088
1000200000 1.4743892982602478
1000300000 0.9770315875388697
1000400000 1.5235611719493105
1000500000 0.2568660436769624
1000600000 0.254785363690569
1000700000 0.25383678828891054
1000800000 0.25303407926450566
1000900000 0.25223900516449144
1001000000 0.2527662465225617
1001100000 0.2515469441693127
1001200000 0.2504933734204544
1001300000 0.25049779194546146
1001400000 0.250585990033102
1001500000 0.24890793030411004
1001600000 0.24997356056522527
1001700000 0.24839179118139

1025200000 0.2807041798322625
1025300000 0.4868322904228476
1025400000 0.239263223886019
1025500000 0.25909408877293666
1025600000 0.3331086555307369
1025700000 0.25895622435212534
1025800000 0.233401807510576
1025900000 0.3465355208538353
1026000000 0.280788150132217
1026100000 0.20170988425302902
1026200000 0.3205690117194606
1026300000 0.2900313384506345
1026400000 0.19163957380781108
1026500000 0.3202015800941037
1026600000 0.2941735422644119
1026700000 0.16812573226960564
1026800000 0.35599691659141186
1026900000 0.2880493721825202
1027000000 0.19391115467378234
1027100000 0.31688841815362173
1027200000 0.2952539304206014
1027300000 0.28471045013870533
1027400000 0.26848396762748494
1027500000 0.3073129837607284
1027600000 0.23170988590051464
1027700000 0.24153080507281757
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.31666836278323934
1027900000 0.2691626319870002
1028000000 0.25083171588262654
1028100000 0.2713270976671219
1028200000 0.271033

best so far: 0
type: [2, 34, 1, 2, 1] 136
cases of this type: 272
best so far: 0
type: [2, 34, 2, 1, 1] 136
cases of this type: 136
best so far: 0
type: [2, 68, 1, 1, 1] 136
cases of this type: 68
best so far: 0
type: [4, 1, 1, 1, 34] 136
cases of this type: 1336336
1045400000 0.0565868979131937
1045500000 0.053854512589758645
1045600000 0.04817273763468199
1045700000 0.04337819428983728
1045800000 0.05287479038057724
1045900000 0.03718012715978556
1046000000 0.029654634473068184
1046100000 0.010730962397181989
1046200000 0.03257407151370976
1046300000 0.031748197888079616
1046400000 0.03488937075669169
1046500000 0.02968013966622088
1046600000 0.02807211875746581
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.034424013835261266
1046800000 0.015271625873763033
1046900000 0.014353904595088295
1047000000 0.011194856188178061
1047100000 0.005435552215288746
1047200000 0.00895034883001579
1047300000 0.009637803039105733
best so far: 0
type: [4, 1, 1, 17, 